/home/sergio/Projects/Doutorado/ML/yolov5
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Uncomment de git clone command if is the firs time
#!git clone https://github.com/ultralytics/yolov5
%cd ../yolov5

/home/sergio/Projects/Doutorado/ML/yolov5


In [4]:
import torch
import utils # from 
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.0-226-g19e0208 torch 1.10.2+cu102 CUDA:0 (NVIDIA GeForce GTX 1650, 3912MiB)


Setup complete ✅ (8 CPUs, 15.5 GB RAM, 90.1/371.3 GB disk)


In [5]:
# test...
!python3 detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images 
# display.Image(filename='runs/detect/exp/zidane.jpg', width=600)

detect: weights=['yolov5s.pt'], source=data/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-226-g19e0208 torch 1.10.2+cu102 CUDA:0 (NVIDIA GeForce GTX 1650, 3912MiB)

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
image 1/2 /home/sergio/Projects/Doutorado/ML/yolov5/data/images/bus.jpg: 640x480 4 persons, 1 bus, Done. (0.015s)
image 2/2 /home/sergio/Projects/Doutorado/ML/yolov5/data/images/zidane.jpg: 384x640 2 persons, 1 tie, Done. (0.013s)
Speed: 0.6ms pre-process, 14.1ms inference, 2.4ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp20


In [7]:
# from yolov5/detect.py
import argparse
import os
import sys
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn

ROOT = Path.cwd()  # YOLOv5 root directory
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative

from models.common import DetectMultiBackend
from utils.datasets import IMG_FORMATS, VID_FORMATS, LoadImages, LoadStreams
from utils.general import (LOGGER, check_file, check_img_size, check_imshow, check_requirements, colorstr,
                           increment_path, non_max_suppression, print_args, scale_coords, strip_optimizer, xyxy2xywh)
from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, time_sync

@torch.no_grad()
def run(weights=ROOT / 'yolov5s.pt',  # model.pt path(s)
        source=ROOT / 'data/images',  # file/dir/URL/glob, 0 for webcam
        data=ROOT / 'data/coco128.yaml',  # dataset.yaml path
        imgsz=(640, 640),  # inference size (height, width)
        conf_thres=0.25,  # confidence threshold
        iou_thres=0.45,  # NMS IOU threshold
        max_det=1000,  # maximum detections per image
        device='',  # cuda device, i.e. 0 or 0,1,2,3 or cpu
        view_img=False,  # show results
        save_txt=False,  # save results to *.txt
        save_conf=False,  # save confidences in --save-txt labels
        save_crop=False,  # save cropped prediction boxes
        nosave=False,  # do not save images/videos
        classes=None,  # filter by class: --class 0, or --class 0 2 3
        agnostic_nms=False,  # class-agnostic NMS
        augment=False,  # augmented inference
        visualize=False,  # visualize features
        update=False,  # update all models
        project=ROOT / 'runs/detect',  # save results to project/name
        name='exp',  # save results to project/name
        exist_ok=False,  # existing project/name ok, do not increment
        line_thickness=3,  # bounding box thickness (pixels)
        hide_labels=False,  # hide labels
        hide_conf=False,  # hide confidences
        half=False,  # use FP16 half-precision inference
        dnn=False,  # use OpenCV DNN for ONNX inference
        ):
    source = str(source)
    save_img = not nosave and not source.endswith('.txt')  # save inference images
    is_file = Path(source).suffix[1:] in (IMG_FORMATS + VID_FORMATS)
    is_url = source.lower().startswith(('rtsp://', 'rtmp://', 'http://', 'https://'))
    webcam = source.isnumeric() or source.endswith('.txt') or (is_url and not is_file)
    if is_url and is_file:
        source = check_file(source)  # download

    # Directories
    save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    # Load model
    device = select_device(device)
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data)
    stride, names, pt, jit, onnx, engine = model.stride, model.names, model.pt, model.jit, model.onnx, model.engine
    imgsz = check_img_size(imgsz, s=stride)  # check image size

    # Half
    half &= (pt or jit or onnx or engine) and device.type != 'cpu'  # FP16 supported on limited backends with CUDA
    if pt or jit:
        model.model.half() if half else model.model.float()

    # Dataloader
    if webcam:
        view_img = check_imshow()
        cudnn.benchmark = True  # set True to speed up constant image size inference
        dataset = LoadStreams(source, img_size=imgsz, stride=stride, auto=pt)
        bs = len(dataset)  # batch_size
    else:
        dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt)
        bs = 1  # batch_size
    vid_path, vid_writer = [None] * bs, [None] * bs

    # Run inference
    model.warmup(imgsz=(1, 3, *imgsz), half=half)  # warmup
    dt, seen = [0.0, 0.0, 0.0], 0

    # for name in names:
    #     LOGGER.info(name)
    t0 = time_sync()
    frame_count = 0
    for path, im, im0s, vid_cap, s in dataset:
        t1 = time_sync()
        im = torch.from_numpy(im).to(device)
        im = im.half() if half else im.float()  # uint8 to fp16/32
        im /= 255  # 0 - 255 to 0.0 - 1.0
        if len(im.shape) == 3:
            im = im[None]  # expand for batch dim
        t2 = time_sync()
        dt[0] += t2 - t1

        # Inference
        visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
        pred = model(im, augment=augment, visualize=visualize)
        t3 = time_sync()
        dt[1] += t3 - t2

        # NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
        dt[2] += time_sync() - t3

        # Second-stage classifier (optional)
        # pred = utils.general.apply_classifier(pred, classifier_model, im, im0s)

        # Process predictions
        for i, det in enumerate(pred):  # per image
            seen += 1
            if webcam:  # batch_size >= 1
                p, im0, frame = path[i], im0s[i].copy(), dataset.count
                s += f'{i}: '
            else:
                p, im0, frame = path, im0s.copy(), getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # im.jpg
            txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # im.txt
            s += '%gx%g ' % im.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            imc = im0.copy() if save_crop else im0  # for save_crop
            annotator = Annotator(im0, line_width=line_thickness, example=str(names))
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(im.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                # Write results
                for *xyxy, conf, cls in reversed(det):
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # label format
                        with open(txt_path + '.txt', 'a') as f:
                            f.write(('%g ' * len(line)).rstrip() % line + '\n')

                    if save_img or save_crop or view_img:  # Add bbox to image
                        c = int(cls)  # integer class
                        label = None if hide_labels else (names[c] if hide_conf else f'{names[c]} {conf:.2f}')
                        annotator.box_label(xyxy, label, color=colors(c, True))
                        if save_crop:
                            save_one_box(xyxy, imc, file=save_dir / 'crops' / names[c] / f'{p.stem}.jpg', BGR=True)

            # Print time (inference-only)
            LOGGER.info(f'{s}Done. ({t3 - t2:.3f}s)')

            # Stream results
            im0 = annotator.result()
            if view_img:
                cv2.imshow(str(p), im0)
                cv2.waitKey(1)  # 1 millisecond

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'image':
                    cv2.imwrite(save_path, im0)
                else:  # 'video' or 'stream'
                    if vid_path[i] != save_path:  # new video
                        vid_path[i] = save_path
                        if isinstance(vid_writer[i], cv2.VideoWriter):
                            vid_writer[i].release()  # release previous video writer
                        if vid_cap:  # video
                            fps = vid_cap.get(cv2.CAP_PROP_FPS)
                            w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                            h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        else:  # stream
                            fps, w, h = 30, im0.shape[1], im0.shape[0]
                        save_path = str(Path(save_path).with_suffix('.mp4'))  # force *.mp4 suffix on results videos
                        vid_writer[i] = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
                    vid_writer[i].write(im0)
            frame_count += 1

    tempo_total = time_sync() - t0
    LOGGER.info(f'Tempo total: {tempo_total:.3f}s - Frames: {frame_count} - {frame_count/tempo_total :.3f} FPS')
    # Print results
    t = tuple(x / seen * 1E3 for x in dt)  # speeds per image
    LOGGER.info(f'Speed: %.1fms pre-process, %.1fms inference, %.1fms NMS per image at shape {(1, 3, *imgsz)}' % t)
    if save_txt or save_img:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
        LOGGER.info(f"Results saved to {colorstr('bold', save_dir)}{s}")
    if update:
        strip_optimizer(weights)  # update model (to fix SourceChangeWarning)

In [10]:
# Model
# model = torch.hub.load('ultralytics/yolov5', 'yolov5')  # or yolov5m, yolov5l, yolov5x, custom

# Images
# img = 'https://ultralytics.com/images/zidane.jpg'  # or file, Path, PIL, OpenCV, numpy, list

# Inference
# results = model(img)

# Results
# results.print()
# results.show()

In [11]:
# python3 detect_speed.py --classes 0 15 16  --source ../material\ de\ teste/video1.mp4
classes = [0, 15, 16] # person, cat, dog

# python3 detect_speed.py --classes 0 15 16  --source ../material\ de\ teste/video1.mp4
run(classes=classes, source="../material de teste/video1.mp4")

YOLOv5 🚀 v6.0-226-g19e0208 torch 1.10.2+cu102 CUDA:0 (NVIDIA GeForce GTX 1650, 3912MiB)

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
video 1/1 (1/1033) /home/sergio/Projects/Doutorado/ML/material de teste/video1.mp4: 384x640 Done. (0.013s)
video 1/1 (2/1033) /home/sergio/Projects/Doutorado/ML/material de teste/video1.mp4: 384x640 Done. (0.013s)
video 1/1 (3/1033) /home/sergio/Projects/Doutorado/ML/material de teste/video1.mp4: 384x640 Done. (0.011s)
video 1/1 (4/1033) /home/sergio/Projects/Doutorado/ML/material de teste/video1.mp4: 384x640 Done. (0.014s)
video 1/1 (5/1033) /home/sergio/Projects/Doutorado/ML/material de teste/video1.mp4: 384x640 Done. (0.015s)
video 1/1 (6/1033) /home/sergio/Projects/Doutorado/ML/material de teste/video1.mp4: 384x640 Done. (0.014s)
video 1/1 (7/1033) /home/sergio/Projects/Doutorado/ML/material de teste/video1.mp4: 384x640 Done. (0.013s)
video 1/1 (8/1033) /home/sergio/Projects/Doutorado/ML/material de teste/video1.mp4: 3

In [12]:
# python3 detect.py --weights yolov5m6.pt --classes 0 15 16  --source ../material\ de\ teste/video1.mp4

run(classes=classes, weights='yolov5m6.pt', source="../material de teste/video1.mp4")

YOLOv5 🚀 v6.0-226-g19e0208 torch 1.10.2+cu102 CUDA:0 (NVIDIA GeForce GTX 1650, 3912MiB)

Fusing layers... 
Model Summary: 378 layers, 35704908 parameters, 0 gradients
video 1/1 (1/1033) /home/sergio/Projects/Doutorado/ML/material de teste/video1.mp4: 384x640 Done. (0.032s)
video 1/1 (2/1033) /home/sergio/Projects/Doutorado/ML/material de teste/video1.mp4: 384x640 Done. (0.025s)
video 1/1 (3/1033) /home/sergio/Projects/Doutorado/ML/material de teste/video1.mp4: 384x640 Done. (0.027s)
video 1/1 (4/1033) /home/sergio/Projects/Doutorado/ML/material de teste/video1.mp4: 384x640 Done. (0.024s)
video 1/1 (5/1033) /home/sergio/Projects/Doutorado/ML/material de teste/video1.mp4: 384x640 Done. (0.024s)
video 1/1 (6/1033) /home/sergio/Projects/Doutorado/ML/material de teste/video1.mp4: 384x640 Done. (0.026s)
video 1/1 (7/1033) /home/sergio/Projects/Doutorado/ML/material de teste/video1.mp4: 384x640 Done. (0.026s)
video 1/1 (8/1033) /home/sergio/Projects/Doutorado/ML/material de teste/video1.mp4: 

In [13]:
# Download COCO val
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')
!unzip -q tmp.zip -d ../datasets && rm tmp.zip

100%|██████████| 780M/780M [02:56<00:00, 4.65MB/s] 


In [3]:
# Run YOLOv5s6 on COCO val
!python3 val.py --weights yolov5s6.pt --data coco.yaml --img 640 --iou 0.65 --half --verbose

val: data=/home/sergio/Projects/Doutorado/ML/yolov5/data/cocopp.yaml, weights=['yolov5s6.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.65, task=val, device=, workers=8, single_cls=False, augment=False, verbose=True, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.0-226-g19e0208 torch 1.10.2+cu102 CUDA:0 (NVIDIA GeForce GTX 1650, 3912MiB)

Fusing layers... 
Model Summary: 280 layers, 12612508 parameters, 0 gradients
val: Scanning '/home/sergio/Projects/Doutorado/ML/datasets/coco/val2017.cache' i
               Class     Images     Labels          P          R     mAP@.5 mAP@
Traceback (most recent call last):
  File "val.py", line 382, in <module>
    main(opt)
  File "val.py", line 355, in main
    run(**vars(opt))
  File "/home/sergio/.local/lib/python3.8/site-packages/torch/autograd/grad_mode.py", line 28, in decorate_context
    return func(*args, **kwargs)
  File "val.p